# Признаки волатильности

- Реализованная волатильность: rolling windows на log_returns
- EWMA-волатильность: экспоненциальное сглаживание
- Parkinson volatility: расчёт на high-low
- Garman-Klass volatility: расчёт на OHLC
- Направленная волатильность: отдельные ап/даун движения

Все расчёты сохраняются структурированно для одного тикера и для всего набора.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


✅ Библиотеки загружены


## Загрузка данных

Используем данные из предыдущего этапа `01_data_loading.ipynb`


In [10]:
# Получаем текущую рабочую директорию
current_dir = Path.cwd()
print(f"Текущая директория: {current_dir}")

# Строим путь к данным относительно текущей директории
# Если ноутбук в 'ML/02_feature_engineering/', а данные в 'ML/data/processed'
DATA_DIR = current_dir.parent.parent / "ML" / "data" / "processed"
print(f"Путь к данным: {DATA_DIR}")

# Проверяем существование пути
if DATA_DIR.exists():
    print("Директория найдена!")
else:
    print("Директория не найдена, проверьте путь")

# Пример загрузки для одного тикера
ticker = 'SBER'
df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")
print(f"Загружено: {len(df)} записей для {ticker}")
print(df.head())


Текущая директория: e:\Python\VolatilityChecker\MOEXScanner\ML\02_feature_engineering
Путь к данным: e:\Python\VolatilityChecker\MOEXScanner\ML\data\processed
Директория найдена!
Загружено: 1301 записей для SBER
     open   close    high     low         value    volume       begin  \
0  205.92  205.03  207.64  204.77  9.481496e+09  46023280  2020-10-13   
1  205.06  207.83  208.00  204.78  9.685023e+09  46840170  2020-10-14   
2  207.46  203.85  207.57  201.95  1.562762e+10  76502030  2020-10-15   
3  204.02  201.17  204.25  200.50  1.339234e+10  66258500  2020-10-16   
4  201.55  202.78  204.00  201.55  9.616800e+09  47421400  2020-10-19   

                   end                date  log_return  
0  2020-10-13 23:59:59 2020-10-13 23:59:59   -0.005593  
1  2020-10-14 23:59:59 2020-10-14 23:59:59    0.013564  
2  2020-10-15 23:59:59 2020-10-15 23:59:59   -0.019336  
3  2020-10-16 23:59:59 2020-10-16 23:59:59   -0.013234  
4  2020-10-19 23:59:59 2020-10-19 23:59:59    0.007971  


## Функции расчета волатильности


In [11]:
def realized_volatility(returns, window=30):
    """Реализованная волатильность: rolling std на log returns"""
    return returns.rolling(window=window).std() * np.sqrt(252)


def ewma_volatility(returns, span=30):
    """EWMA волатильность: экспоненциальное сглаживание"""
    return returns.ewm(span=span).std() * np.sqrt(252)


def parkinson_volatility(high, low, window=30):
    """Parkinson volatility: на основе high-low"""
    hl_ratio = np.log(high / low) ** 2
    return np.sqrt(hl_ratio.rolling(window=window).mean() / (4 * np.log(2))) * np.sqrt(252)


def garman_klass_volatility(open_price, high, low, close, window=30):
    """Garman-Klass volatility: использует OHLC"""
    hl = np.log(high / low) ** 2
    co = np.log(close / open_price) ** 2
    gk = 0.5 * hl - (2 * np.log(2) - 1) * co
    return np.sqrt(gk.rolling(window=window).mean()) * np.sqrt(252)


def directional_volatility(returns, window=30):
    """Направленная волатильность: отдельно вверх и вниз"""
    up_vol = returns[returns > 0].rolling(window=window).std() * np.sqrt(252)
    down_vol = returns[returns < 0].rolling(window=window).std() * np.sqrt(252)
    return up_vol, down_vol

print("✅ Функции волатильности загружены")


✅ Функции волатильности загружены


## Расчет признаков волатильности


In [ ]:
# Расчет всех типов волатильности
windows = [10, 30, 60]

for window in windows:
    df[f'realized_vol_{window}'] = realized_volatility(df['log_return'], window=window)
    df[f'ewma_vol_{window}'] = ewma_volatility(df['log_return'], span=window)
    df[f'parkinson_vol_{window}'] = parkinson_volatility(df['high'], df['low'], window=window)
    df[f'gk_vol_{window}'] = garman_klass_volatility(df['open'], df['high'], df['low'], df['close'], window=window)
    up, down = directional_volatility(df['log_return'], window=window)
    df[f'up_vol_{window}'] = up
    df[f'down_vol_{window}'] = down

print("✅ Признаки волатильности рассчитаны")
print(f"\nСтолбцы: {df.columns.tolist()}")
print(f"\nПример данных:")
print(df[['date', 'close', 'realized_vol_30', 'ewma_vol_30', 'parkinson_vol_30, gk_vol_30']].tail())


✅ Признаки волатильности рассчитаны

Столбцы: ['open', 'close', 'high', 'low', 'value', 'volume', 'begin', 'end', 'date', 'log_return', 'realized_vol_10', 'ewma_vol_10', 'parkinson_vol_10', 'gk_vol_10', 'realized_vol_30', 'ewma_vol_30', 'parkinson_vol_30', 'gk_vol_30', 'realized_vol_60', 'ewma_vol_60', 'parkinson_vol_60', 'gk_vol_60']

Пример данных:
                    date   close  realized_vol_30  ewma_vol_30  \
1296 2025-10-07 23:59:54  293.89         0.167693     0.182651   
1297 2025-10-08 23:59:51  281.90         0.203524     0.241398   
1298 2025-10-09 23:59:55  289.37         0.220255     0.260294   
1299 2025-10-10 23:59:55  285.12         0.221308     0.257650   
1300 2025-10-11 15:00:27  284.44         0.221024     0.249209   

      parkinson_vol_30  
1296          0.174994  
1297          0.203165  
1298          0.215346  
1299          0.217627  
1300          0.217960  


## Сохранение результатов


In [ ]:
# Сохранение с признаками волатильности
OUTPUT_DIR = Path('data') / 'features'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_path = OUTPUT_DIR / f"{ticker}_volatility_features.parquet"
df.to_parquet(output_path, index=False)
print(f"✅ Сохранено: {output_path}")
